In [1]:
!pip install torch torchvision

Defaulting to user installation because normal site-packages is not writeable
distutils: /home/abaxter/.local/lib/python3.9/site-packages
sysconfig: /home/abaxter/.local/lib64/python3.9/site-packages
user = True
home = None
root = None
prefix = None


In [2]:
!pip install scikit-learn

Defaulting to user installation because normal site-packages is not writeable
distutils: /home/abaxter/.local/lib/python3.9/site-packages
sysconfig: /home/abaxter/.local/lib64/python3.9/site-packages
user = True
home = None
root = None
prefix = None


In [1]:
import torch
import torch.nn as nn
from torchvision import models
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from torchvision.transforms.functional import to_pil_image
from PIL import Image
import os
import pandas as pd
import numpy as np

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [3]:
class CSVDataset(Dataset):
    def __init__(self, dataframe, image_root_dir, target_columns=None, transform=None,
                 save_dir=None, use_saved_images=False):
        self.data = dataframe
        self.image_root_dir = image_root_dir
        self.target_columns = target_columns
        self.transform = transform
        self.save_dir = save_dir
        self.use_saved_images = use_saved_images

        if self.save_dir:
            os.makedirs(self.save_dir, exist_ok=True)

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        row = self.data.iloc[idx]
        
        # Use index for the saved tensor filename
        #saved_image_path = os.path.join(self.save_dir, f"{idx}.pt")
        image_index = row['Unnamed: 0']
        saved_image_path = os.path.join(self.save_dir, f"{image_index}.pt")



        if self.use_saved_images:
            if os.path.exists(saved_image_path):
                image_tensor = torch.load(saved_image_path)
            else:
                raise FileNotFoundError(f"Saved tensor not found: {saved_image_path}")
        else:
            original_image_path = os.path.join(self.image_root_dir, row['Path'])
            image = Image.open(original_image_path).convert("L")
            image_tensor = self.transform(image) if self.transform else transforms.ToTensor()(image)

            if self.save_dir:
                torch.save(image_tensor, saved_image_path)

        if self.target_columns:
            labels = pd.to_numeric(row[self.target_columns], errors='coerce').fillna(0).astype(float).values
            labels = torch.tensor(labels, dtype=torch.float32)
            return image_tensor, labels

        return image_tensor

In [4]:
class MultiLabelResNet50(nn.Module):
    def __init__(self, num_classes):
        super(MultiLabelResNet50, self).__init__()
        
        # Load pre-trained ResNet50
        self.base_model = models.resnet50(weights=models.ResNet50_Weights.IMAGENET1K_V1)
        
        # Modify the fully connected layer for multi-label classification
        self.base_model.fc = nn.Sequential(
            nn.Linear(self.base_model.fc.in_features, 512),  # New intermediate layer
            nn.ReLU(),
            nn.Dropout(0.5),  # Dropout to prevent overfitting
            nn.Linear(512, num_classes),  # Output layer
            #nn.Sigmoid()  # Sigmoid for multi-label classification
            nn.Tanh()  #This is between -1 and 1

           # nn.Linear(self.base_model.fc.in_features, num_classes),
           # nn.Sigmoid()  # Sigmoid activation for multi-label classification
        )

    def forward(self, x):
        return self.base_model(x)

In [5]:
image_root = '/central/groups/CS156b/2025/CodeMonkeys/input_images'
image_root_dir = "train"
train_save_dir = os.path.join(image_root, 'train')
#test_save_dir = os.path.join(image_root, 'test')
full_train_df = pd.read_csv('train2023.csv')
#filtered_train_df = full_train_df.iloc[:29692]
filtered_train_df = full_train_df.iloc[:15000]



In [6]:
from sklearn.model_selection import train_test_split

# Define your target columns once
target_columns = [
    "No Finding", "Enlarged Cardiomediastinum", "Cardiomegaly", "Lung Opacity",
    "Pneumonia", "Pleural Effusion", "Pleural Other", "Fracture", "Support Devices"
]

# Step 1: Split the dataframe
train_df, val_df = train_test_split(filtered_train_df, test_size=0.15, random_state=42)

# Step 2: Create training dataset
train_dataset = CSVDataset(
    dataframe=train_df, 
    image_root_dir=image_root, 
    target_columns=target_columns, 
    save_dir=train_save_dir, 
    use_saved_images=True
)

# Step 3: Create validation dataset
val_dataset = CSVDataset(
    dataframe=val_df, 
    image_root_dir=image_root, 
    target_columns=target_columns, 
    save_dir=train_save_dir, 
    use_saved_images=True
)

# Step 4: Create DataLoaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

In [7]:
def freeze_base_layers(model, until_layer=6):
    """
    Freeze layers of ResNet-50 up to a certain stage (e.g., until_layer=6 means keep layers 0-5 frozen).
    """
    child_counter = 0
    for child in model.base_model.children():
        if child_counter < until_layer:
            for param in child.parameters():
                param.requires_grad = False
        child_counter += 1
    return model


In [8]:
class_weights = {}

# Loop over each target column
for col in target_columns:
    # Count the occurrences of each class in the column
    counts = filtered_train_df[col].value_counts()
    total = len(filtered_train_df[col])
    
    # Calculate class weights using inverse frequency (you can also experiment with other strategies)
    weights = {
        -1: total / (counts.get(-1, 0) + 1),  # Add 1 to avoid division by zero
        0: total / (counts.get(0, 0) + 1),
        1: total / (counts.get(1, 0) + 1)
    }
    
    # Store weights for each class
    class_weights[col] = weights

# Example: Print out the weights for each class
for col in target_columns:
    print(f"Class weights for {col}: {class_weights[col]}")


Class weights for No Finding: {-1: np.float64(1.241105411219593), 0: np.float64(14.245014245014245), 1: np.float64(8.055853920515574)}
Class weights for Enlarged Cardiomediastinum: {-1: np.float64(7.932310946589106), 0: np.float64(14.164305949008499), 1: np.float64(22.900763358778626)}
Class weights for Cardiomegaly: {-1: np.float64(11.415525114155251), 0: np.float64(19.329896907216494), 1: np.float64(8.620689655172415)}
Class weights for Lung Opacity: {-1: np.float64(31.315240083507305), 0: np.float64(32.467532467532465), 1: np.float64(2.2765214751859157)}
Class weights for Pneumonia: {-1: np.float64(63.29113924050633), 0: np.float64(10.060362173038229), 1: np.float64(69.76744186046511)}
Class weights for Pleural Effusion: {-1: np.float64(5.211952744961779), 0: np.float64(16.286644951140065), 1: np.float64(2.886836027713626)}
Class weights for Pleural Other: {-1: np.float64(1363.6363636363637), 0: np.float64(78.125), 1: np.float64(45.04504504504504)}
Class weights for Fracture: {-1: n

In [9]:
criterion = nn.MSELoss(reduction='none')

def masked_MSE_loss(output, target, class_weights):
    # Create a mask for non-NaN target values
    mask = ~torch.isnan(target)
    
    # Apply the MSE loss
    loss = criterion(output, target)
    
    # Loop through each class and apply the class weights
    for class_idx, col in enumerate(target_columns):
        # Get the weight for the current class (0, -1, or 1)
        class_values = target[:, class_idx].cpu().numpy()  # Get the class values for the current class
        
        # Apply the class weights to each class value
        weight = np.vectorize(lambda x: class_weights[col].get(x, 1))(class_values)  # Default to weight of 1 for unknown values
        
        # Convert the weights to a torch tensor and ensure the shape matches
        weight = torch.tensor(weight, dtype=torch.float32, device=output.device).view(-1, 1)
        
        # Apply the weight to the loss (broadcast the weight to match the loss shape)
        loss[mask[:, class_idx]] *= weight.squeeze()

    return loss[mask].mean()

#added weights now
'''criterion = nn.MSELoss(reduction='none')

def masked_MSE_loss(output, target):
    mask = ~torch.isnan(target)
    loss = criterion(output, target)
    return loss[mask].mean()
'''

'''#This one better apparently?
criterion = nn.BCEWithLogitsLoss(reduction='none')  # No reduction yet

def masked_BCE_loss(output, target):
    # Create mask for non-NaN targets
    mask = ~torch.isnan(target)  # Same shape as output/target

    # Compute unreduced BCE loss
    loss = criterion(output, target)

    # Apply mask and compute mean over valid entries
    masked_loss = loss[mask]
    return masked_loss.mean()'''


"#This one better apparently?\ncriterion = nn.BCEWithLogitsLoss(reduction='none')  # No reduction yet\n\ndef masked_BCE_loss(output, target):\n    # Create mask for non-NaN targets\n    mask = ~torch.isnan(target)  # Same shape as output/target\n\n    # Compute unreduced BCE loss\n    loss = criterion(output, target)\n\n    # Apply mask and compute mean over valid entries\n    masked_loss = loss[mask]\n    return masked_loss.mean()"

In [13]:
import torch.optim as optim

num_classes = 9  # Adjust based on your number of labels
model = MultiLabelResNet50(num_classes=num_classes).to(device)#, freeze_base=False).to(device)
model = freeze_base_layers(model, until_layer=6)  # Experiment with values from 5 to 7

# Loss function and optimizer

optimizer = optim.Adam(model.parameters(), lr=0.001) #Could try 0.0001

# Training Loop
num_epochs = 30
for epoch in range(num_epochs):
    model.train()  # Set the model to training mode
    running_loss = 0.0
    correct = 0
    total = 0
    
    for batch_idx, (images, labels) in enumerate(train_loader):
        images, labels = images.to(device), labels.to(device)
        
        optimizer.zero_grad()  # Zero the gradients
        
        # Forward pass
        outputs = model(images)
        
        # Compute loss
        loss = masked_MSE_loss(outputs, labels, class_weights)
        running_loss += loss.item()
        
        # Backward pass and optimization
        loss.backward()
        optimizer.step()
        
        # Optional: compute accuracy metrics
        '''predicted = torch.sigmoid(outputs) > 0.5  # Multi-label classification
        correct += (predicted == labels).sum().item()
        total += labels.numel()'''
        predicted = outputs 

        # Create predictions based on thresholds
        predicted_class = torch.where(predicted > 0.5, torch.tensor(1.0), 
                                    torch.where(predicted < -0.5, torch.tensor(-1.0), torch.tensor(0.0)))
        # Compare predictions with labels
        correct += (predicted_class == labels).sum().item()
        total += labels.numel()


    avg_loss = running_loss / len(train_loader)
    accuracy = correct / total
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {avg_loss:.4f}, Accuracy: {accuracy:.4f}")
    
    # Optionally, save the model
    if (epoch+1)%10 == 0:
        torch.save(model.state_dict(), f"models/model2_epoch_{epoch+1}.pth")




RuntimeError: The size of tensor a (9) must match the size of tensor b (32) at non-singleton dimension 1

## Testing Model on Validation Set

In [ ]:
from sklearn.metrics import f1_score, precision_score, recall_score, accuracy_score
import numpy as np
import torch

def evaluate_model_per_class(val_loader, device, model_path):
    model = MultiLabelResNet50(num_classes=9).to(device)
    model.load_state_dict(torch.load(model_path, map_location=device))
    model.eval()

    all_preds = []
    all_labels = []
    
    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            predicted = torch.tanh(outputs)  # Use tanh to map to the range [-1, 1]
            
            # Convert predictions to {-1, 0, 1}
            predicted_class = torch.where(predicted > 0.5, torch.tensor(1.0).to(device),
                                          torch.where(predicted < -0.5, torch.tensor(-1.0).to(device), torch.tensor(0.0).to(device)))
            
            all_preds.append(predicted_class.cpu().numpy())
            all_labels.append(labels.cpu().numpy())
    
    # Convert lists to numpy arrays
    all_preds = np.concatenate(all_preds, axis=0)
    all_labels = np.concatenate(all_labels, axis=0)
    
    # Compute metrics per class, ignoring NaN values for each class
    for class_idx in range(all_preds.shape[1]):  # Iterate through each class
        # Mask NaN values (we assume NaN might be in the true labels)
        mask = ~np.isnan(all_labels[:, class_idx])
        
        # Filter out NaN values
        filtered_preds = all_preds[mask, class_idx]
        filtered_labels = all_labels[mask, class_idx]
        
        if len(filtered_labels) == 0:  # If no valid data for this class
            print(f"Class {class_idx} has no valid data")
            continue
        
        # Calculate metrics for the current class
        f1 = f1_score(filtered_labels, filtered_preds, average='macro', zero_division=1)  # Handle undefined metrics
        precision = precision_score(filtered_labels, filtered_preds, average='macro', zero_division=1)  # Handle undefined metrics
        recall = recall_score(filtered_labels, filtered_preds, average='macro', zero_division=1)  # Handle undefined metrics
        accuracy = accuracy_score(filtered_labels, filtered_preds)

        # Print individual metrics for each class
        print(f"Class {class_idx} - F1 Score: {f1:.4f}, Precision: {precision:.4f}, Recall: {recall:.4f}, Accuracy: {accuracy:.4f}")

# Example usage
evaluate_model_per_class(val_loader, device, "models/model2_epoch_20.pth")


Class 0 - F1 Score: 0.4800, Precision: 0.5587, Recall: 0.4839, Accuracy: 0.8009
Class 1 - F1 Score: 0.3637, Precision: 0.7832, Recall: 0.3647, Accuracy: 0.8409
Class 2 - F1 Score: 0.2972, Precision: 0.6016, Recall: 0.3331, Accuracy: 0.8044
Class 3 - F1 Score: 0.3945, Precision: 0.7229, Recall: 0.3995, Accuracy: 0.5867
Class 4 - F1 Score: 0.3288, Precision: 0.9911, Recall: 0.3333, Accuracy: 0.9733
Class 5 - F1 Score: 0.5778, Precision: 0.5873, Recall: 0.5713, Accuracy: 0.5960
Class 6 - F1 Score: 0.3295, Precision: 0.9924, Recall: 0.3333, Accuracy: 0.9773
Class 7 - F1 Score: 0.3229, Precision: 0.9797, Recall: 0.3333, Accuracy: 0.9391
Class 8 - F1 Score: 0.4514, Precision: 0.7977, Recall: 0.4582, Accuracy: 0.6809


In [ ]:
import torch
from collections import Counter
import numpy as np

def print_label_distribution(val_loader):
    all_labels = []

    for _, labels in val_loader:
        all_labels.append(labels)

    all_labels = torch.cat(all_labels, dim=0).cpu().numpy()  # shape: (num_samples, num_classes)

    num_classes = all_labels.shape[1]

    print("Label distribution per class:")
    for i in range(num_classes):
        unique, counts = np.unique(all_labels[:, i], return_counts=True)
        dist = dict(zip(unique, counts))
        print(f"Class {i}: {dist}")
print_label_distribution(val_loader)

Label distribution per class:
Class 0: {np.float32(-1.0): np.int64(1829), np.float32(0.0): np.int64(169), np.float32(1.0): np.int64(252)}
Class 1: {np.float32(-1.0): np.int64(258), np.float32(0.0): np.int64(1891), np.float32(1.0): np.int64(101)}
Class 2: {np.float32(-1.0): np.int64(192), np.float32(0.0): np.int64(1811), np.float32(1.0): np.int64(247)}
Class 3: {np.float32(-1.0): np.int64(56), np.float32(0.0): np.int64(1188), np.float32(1.0): np.int64(1006)}
Class 4: {np.float32(-1.0): np.int64(34), np.float32(0.0): np.int64(2190), np.float32(1.0): np.int64(26)}
Class 5: {np.float32(-1.0): np.int64(409), np.float32(0.0): np.int64(1045), np.float32(1.0): np.int64(796)}
Class 6: {np.float32(-1.0): np.int64(3), np.float32(0.0): np.int64(2199), np.float32(1.0): np.int64(48)}
Class 7: {np.float32(-1.0): np.int64(47), np.float32(0.0): np.int64(2113), np.float32(1.0): np.int64(90)}
Class 8: {np.float32(-1.0): np.int64(50), np.float32(0.0): np.int64(1164), np.float32(1.0): np.int64(1036)}


In [ ]:
# Create dataset
test_dataset = CSVDataset(
    dataframe=df_first10rows_test, 
    image_root_dir=image_root, 
    target_columns=None, 
    transform=image_transforms,  # Pass the transform
    save_dir=test_save_dir, 
    use_saved_images=False  # Set to True if you want to load tensors from CSV
)

# Create DataLoader
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=True)

# Iterate through batches
for batch_idx, (images) in enumerate(test_loader):
    print(f"Batch {batch_idx + 1}")
    print("Images shape:", images.shape)

## Run Model

In [11]:
model = MultiLabelResNet50(num_classes=9).to(device)
model.load_state_dict(torch.load('models/model2_epoch_20.pth'))
model.eval()  

MultiLabelResNet50(
  (base_model): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (downsample): Sequenti

In [ ]:
test_dir = 'input_images/test'

# List to hold predictions
predictions = []

# Iterate over the test set
for filename in os.listdir(test_dir):
    if filename.endswith(".pt"):  # Only process .pt files
        # Load the image tensor (e.g., ID.pt)
        image_path = os.path.join(test_dir, filename)
        image_tensor = torch.load(image_path).to(device)
        image_tensor = image_tensor.unsqueeze(0)
        
        # Make the prediction
        with torch.no_grad():
            output = model(image_tensor)  # Get model output (size: [1, num_classes])
        
        # Apply thresholding and collect the predictions
        output = output.squeeze().cpu().numpy()  # Remove batch dimension and move to CPU
        
        # Format the output with values between -1 and 1
        predictions.append([filename.split('.')[0]] + list(output))

# Convert predictions to a pandas DataFrame
columns = ["Id", "No Finding", "Enlarged Cardiomediastinum", "Cardiomegaly", "Lung Opacity", 
           "Pneumonia", "Pleural Effusion", "Pleural Other", "Fracture", "Support Devices"]
df_predictions = pd.DataFrame(predictions, columns=columns)

# Save the predictions to a CSV file
df_predictions.to_csv('test_predictions.csv', index=False)

print("Predictions saved to 'test_predictions.csv'")


KeyboardInterrupt: 